In [2]:
# Assignment title: Credit information learning
# Objectives:
# Perform machine learning on something close to real data
# --------------------------------------------------------

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

def Encoder(df):
    columnsToEncode = list(df.select_dtypes(include=['category','object']))
    le = LabelEncoder()
    for feature in columnsToEncode:
        try:
            df[feature] = le.fit_transform(df[feature])
        except:
            print('Error encoding '+feature)
    return df

# Problem 1: Confirmation of competition contents
# -----------------------------------------------

'''
What to learn and what to predict?
    from features, i.e. gender, education, age, income type, classify repayment abilities. (if 0 can't pay, 1 can pay)
What kind of file to create and submit to Kaggle?
    Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.
    For each SK_ID_CURR in the test set, you must predict a probability for the TARGET variable. 
    The file should contain a header and have the following format:
        SK_ID_CURR,TARGET
        100001,0.1
        100005,0.9
        100013,0.2 etc.
What kind of index value will be evaluated for submitted items?
        Probability
'''


# Problem 2: Learning and verification
# ------------------------------------

# data preparation
application_train = pd.read_csv('application_train.csv')
application_test = pd.read_csv('application_test.csv')
print("Number of samples in application_train:{}".format(len(application_train)))
print("Number of features in application_train:{}".format(len(application_train.columns)))
print("Number of samples in application_test:{}".format(len(application_test)))
print("Number of features in application_test:{}".format(len(application_test.columns)))

print(application_test.columns)

application_train.dropna(axis = 1, how='any', inplace=True)
y_train = application_train['TARGET']
application_train = application_train.drop("TARGET", axis=1)
print("after drop features with NA or null values from application_train:{}".format(len(application_train)))
print("features used in train:{}".format(application_train.columns))
print('Feature data types:{}'.format(application_train.dtypes))

application_test = application_test[application_train.columns]
print(application_train.head(10))



application_train = Encoder(application_train)
application_test = Encoder(application_test)

application_train = pd.get_dummies(application_train)
application_test = pd.get_dummies(application_test)
# training 
scaler = StandardScaler()
scaler.fit(application_train)
X_train = scaler.transform(application_train)
X_test = scaler.transform(application_test)

x_tr, x_val, y_tr, y_val = train_test_split(X_train, y_train, random_state=123)

lr = LogisticRegression(C=0.0001)
lr.fit(x_tr, y_tr)

# Problem 3: Estimate for test data
# ---------------------------------

y_test_pred = lr.predict_proba(x_val)[:, 1]
print("roc auc score", roc_auc_score(y_val, y_test_pred))

y_test_pred = lr.predict_proba(X_test)[:, 1]
data = application_test[["SK_ID_CURR"]]
data['TARGET'] = y_test_pred.tolist()
data.to_csv('submission.csv', index=False)
print(data)

print("BEST SCORE: 0.64511  V2")

# Problem 4: Feature engineering
# ------------------------------

application_train = pd.read_csv('application_train.csv')
application_test = pd.read_csv('application_test.csv')
print("Number of samples in application_train:{}".format(len(application_train)))
print("Number of features in application_train:{}".format(len(application_train.columns)))
print("Number of samples in application_test:{}".format(len(application_test)))
print("Number of features in application_test:{}".format(len(application_test.columns)))


correlations = application_train.corr()["TARGET"].sort_values()

print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

'''
From above result, We can see that the most correlated variables are DAYS_BIRTH, EXT_SOURCE_3, EXT_SOURCE_2 and EXT_SOURCE_1.     
'''

ext_data = application_train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
print(ext_data_corrs)


y_ext = ext_data['TARGET']
X_train = ext_data.drop(columns=['TARGET'])
X_test = application_test[X_train.columns]

print(X_train.columns)

imputer = SimpleImputer(strategy="median")
X_train = imputer.fit_transform(X_train)
X_test = imputer.fit_transform(X_test)

scaler = StandardScaler()
scaler.fit(X_train)
train = scaler.transform(X_train)
test = scaler.transform(X_test)


x_tr, x_val, y_tr, y_val = train_test_split(train, y_ext, random_state=123)

log_reg = LogisticRegression(C=0.0001)
log_reg.fit(x_tr, y_tr)

log_reg_pred = log_reg.predict_proba(x_val)[:, 1]
print("roc auc score", roc_auc_score(y_val, log_reg_pred))

y_test_pred = log_reg.predict_proba(X_test)[:, 1]

data = application_test[["SK_ID_CURR"]]
data['TARGET'] = y_test_pred.tolist()
data.to_csv('submission.csv', index=False)
print(data)

print("BEST SCORE: 0.71809206308055")





Number of samples in application_train:307511
Number of features in application_train:122
Number of samples in application_test:48744
Number of features in application_test:121
Index(['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=121)
after drop features with NA or null values from application_train:307511
features used in train:Index(['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE'

roc auc score 0.6520682025451561
       SK_ID_CURR    TARGET
0          100001  0.061083
1          100005  0.101110
2          100013  0.051799
3          100028  0.074639
4          100038  0.116959
...           ...       ...
48739      456221  0.084149
48740      456222  0.095559
48741      456223  0.070960
48742      456224  0.101302
48743      456250  0.083266

[48744 rows x 2 columns]
BEST SCORE: 0.64511  V2


C:\Users\Mohammed Khalil\AppData\Local\Temp\ipykernel_19960\1985496645.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TARGET'] = y_test_pred.tolist()


Number of samples in application_train:307511
Number of features in application_train:122
Number of samples in application_test:48744
Number of features in application_test:121


C:\Users\Mohammed Khalil\AppData\Local\Temp\ipykernel_19960\1985496645.py:116: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlations = application_train.corr()["TARGET"].sort_values()


Most Positive Correlations:
 DEF_60_CNT_SOCIAL_CIRCLE       0.031276
DEF_30_CNT_SOCIAL_CIRCLE       0.032248
LIVE_CITY_NOT_WORK_CITY        0.032518
OWN_CAR_AGE                    0.037612
DAYS_REGISTRATION              0.041975
FLAG_DOCUMENT_3                0.044346
REG_CITY_NOT_LIVE_CITY         0.044395
FLAG_EMP_PHONE                 0.045982
REG_CITY_NOT_WORK_CITY         0.050994
DAYS_ID_PUBLISH                0.051457
DAYS_LAST_PHONE_CHANGE         0.055218
REGION_RATING_CLIENT           0.058899
REGION_RATING_CLIENT_W_CITY    0.060893
DAYS_BIRTH                     0.078239
TARGET                         1.000000
Name: TARGET, dtype: float64

Most Negative Correlations:
 EXT_SOURCE_3                 -0.178919
EXT_SOURCE_2                 -0.160472
EXT_SOURCE_1                 -0.155317
DAYS_EMPLOYED                -0.044932
FLOORSMAX_AVG                -0.044003
FLOORSMAX_MEDI               -0.043768
FLOORSMAX_MODE               -0.043226
AMT_GOODS_PRICE              -0.039645


C:\Users\Mohammed Khalil\AppData\Local\Temp\ipykernel_19960\1985496645.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TARGET'] = y_test_pred.tolist()
